In [1]:
from skimage import io
from os import path, listdir
import numpy as np
import tifffile
import csv
import numpy as np

In [28]:
dir_path = "/media/ula/DATADRIVE1/fos_gfp_tmaze/ctx_landmark/despeckle/"
res_dir_path = dir_path + "alignment_result/"
path_for_icy = res_dir_path + "aligned_despeckle/"

ext = ".tif"
search_window = 20;
stack_window = 9;
substack_size = 5;

max_regions_no = 4;
filename_template = "m{}r{}_{}_spots"
filename_raw_temp = "m{}r{}_{}"
resfilename_template = "m{}r{}"

In [3]:
def find_optimal_substack_trans(substack1, substack2):
    minv = 5000
    minx = 0
    miny = 0
    for x in range(-search_window, search_window):
        for y in range(-search_window, search_window):
            xdif = min(substack1.shape[1], substack2.shape[1])-abs(x);
            ydif = min(substack1.shape[2], substack2.shape[2])-abs(y);
            sub1_tmp = substack1[:,max(0,x):max(0,x)+xdif,max(0,y):max(0,y)+ydif]
            sub2_tmp = substack2[:,max(0,-x):max(0,-x)+xdif,max(0,-y):max(0,-y)+ydif]
            dif_tmp = abs(sub1_tmp - sub2_tmp)
            tmp_avg = np.average(dif_tmp)
            if tmp_avg < minv:
                minv = tmp_avg
                minx = x
                miny = y
    return np.array([[minx],[miny]]), minv

In [4]:
def find_stack_translations(stack1, stack2):
    translations = np.array([[],[]])
    stack_size = stack1.shape[0]
    
    sum_of_avgs = 0
    
    start_idx = 0
    while start_idx + substack_size < stack_size:
        stop_idx = start_idx + substack_size
        if stop_idx + substack_size > stack_size:
            stop_idx = stack_size+1
        coords, sub_avg = find_optimal_substack_trans(stack1[start_idx:stop_idx], stack2[start_idx:stop_idx])
        sum_of_avgs += sub_avg
        translations = np.append(translations, coords, axis = 1)
        start_idx = stop_idx
    return translations, sum_of_avgs

In [5]:
def find_alignment_translations(mouse, region, starting_session_id):
    sn = session_order[starting_session_id-1:starting_session_id+1]
    
    fn1 = dir_path+filename_template.format(mouse, region,sn[0]) +ext
    fn2 = dir_path+filename_template.format(mouse, region,sn[1]) +ext
    
    print(fn1, path.exists(fn1))
    
    if path.exists(fn1) and path.exists(fn2):
        orig1 = io.imread(fn1).astype("uint16")
        orig2 = io.imread(fn2).astype("uint16")

        z1 = orig1.shape[0]
        z2 = orig2.shape[0]
        optimal_sum_of_avgs = 50000
        optimal_translations = []
        minz = 0
        for z in range(-stack_window, stack_window):
            tst1 = orig1[max(0,z) : min(z1, z2+z)];
            tst2 = orig2[max(0,-z) : min(z1-z, z2)];

            coords_list, sum_of_avgs = find_stack_translations(tst1, tst2)
            if sum_of_avgs < optimal_sum_of_avgs:
                optimal_sum_of_avgs = sum_of_avgs
                optimal_translations = coords_list
                minz = z
        aligned1, aligned2 = align_stacks(orig1, orig2, optimal_translations, minz, start_img_id)
        save_results(aligned1, aligned2, optimal_translations, minz, 
                     starting_session_id, resfilename_template.format(mouse, region),res_dir_path)
        raw1 = dir_path+filename_raw_temp.format(mouse, region,sn[0]) +ext
        raw2 = dir_path+filename_raw_temp.format(mouse, region,sn[1]) +ext
        raw1 = io.imread(raw1)
        raw2 = io.imread(raw2)
        aligned1_raw, aligned2_raw = align_stacks(raw1, raw2, optimal_translations, minz, start_img_id)
        save_results(aligned1_raw, aligned2_raw, optimal_translations, minz, starting_session_id, 
                     resfilename_template.format(mouse, region), path_for_icy, to_csv = False)

In [6]:
def calc_adj_translation(max_tr, curr_trans, direction):
    #direction - jesli sesja subsequent to -1
    #do sprawdzenia nawiasy cuda wianki
    return int(max(0, (max(0, max_tr * direction) + max_tr-curr_trans)))

In [7]:
def align_stacks(orig1, orig2, optimal_translations, minz, start_img_id):
    z1 = orig1.shape[0]
    z2 = orig2.shape[0]
    res1 = orig1[max(0,minz) : min(z1, z2+minz)];
    res2 = orig2[max(0,-minz) : min(z1-minz, z2)];

    print("optimal trans ", optimal_translations)
    print("minz ", minz)
    
    max_transx = int(max(optimal_translations[0], key = abs))
    max_transy = int(max(optimal_translations[1], key = abs))

    difx = min(res1.shape[1], res2.shape[1]) - abs(max_transx)
    dify = min(res1.shape[2], res2.shape[2]) - abs(max_transy)

    ret1 = np.empty((res1.shape[0], difx, dify))
    ret2 = np.empty_like(ret1)
    x0 = [calc_adj_translation(max_transx,x, -1) for x in optimal_translations[0]]
    y0 = [calc_adj_translation(max_transy,y, -1) for y in optimal_translations[1]]
    x0_b = max(0, max_transx)
    y0_b = max(0, max_transy)
    for idx, (r1, r2) in enumerate(zip(res1,res2)):
        tr_idx = idx//substack_size
        if tr_idx >= len(x0):
            tr_idx = len(x0) - 1
        ret1[idx] =  r1[x0_b:x0_b+difx,y0_b:y0_b+dify]
        ret2[idx] =  r2[x0[tr_idx]:x0[tr_idx]+difx,y0[tr_idx]:y0[tr_idx]+dify]
    return ret1, ret2


In [32]:
def prepare_for_tif_save(image):
    newimg = np.empty((image.shape[0], 1, image.shape[1], image.shape[2]))
    for idx, im in enumerate(image):
        newimg[idx] = np.array([im])
    return newimg.astype("uint8")

In [25]:
def save_results(ret1, ret2, optimal_translations, minz, start_img_id, namebase, res_dir, to_csv=True):
    ret1 = prepare_for_tif_save(ret1)
    ret2 = prepare_for_tif_save(ret2)
    findif = abs(ret1-ret2)

    tifffile.imwrite(res_dir+namebase+"_"+str(start_img_id)+suff+ext,ret1.astype('uint8'),imagej=True, metadata={'unit': 'pixels','axes': 'ZCYX'})
    tifffile.imwrite(res_dir+namebase+"_"+str(start_img_id+1)+suff+ext, ret2.astype('uint8'),imagej=True, metadata={'unit': 'pixels','axes': 'ZCYX'})
    tifffile.imwrite(res_dir+namebase+"_"+suff+ext,findif.astype('uint8'),imagej=True, metadata={'unit': 'pixels','axes': 'ZCYX'})
    
    if to_csv:
        with open(res_dir+namebase+"_"+str(start_img_id)+suff+ '.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(optimal_translations[0])
            writer.writerow(optimal_translations[1])
            writer.writerow(str(minz))

In [9]:
landmark_first_mice = [1, 3, 4, 6,7,9,12,15,17, 18]
session_order = ["landmark", "ctx1", "ctx2"]


for m in landmark_first_mice:
    for i in range(1, max_regions_no+1):
        for start_img_id in [1,2]:
            suff = "_" + session_order[start_img_id-1] + "_" + session_order[start_img_id]
            find_alignment_translations(m, i, start_img_id)

/media/ula/DATADRIVE1/fos_gfp_tmaze/ctx_landmark/despeckle/m1r1_landmark_spots.tif True
optimal trans  [[15. 13. 11.  9.  8.  8.  8.  7.  7.  6.  6.  5.  3.]
 [ 0. -2. -2. -3. -3. -3. -4. -4. -4. -4. -4. -5. -5.]]
minz  3
optimal trans  [[15. 13. 11.  9.  8.  8.  8.  7.  7.  6.  6.  5.  3.]
 [ 0. -2. -2. -3. -3. -3. -4. -4. -4. -4. -4. -5. -5.]]
minz  3
/media/ula/DATADRIVE1/fos_gfp_tmaze/ctx_landmark/despeckle/m1r1_ctx1_spots.tif True
optimal trans  [[ 5.  5.  6.  6.  5.  4.  3.  2.  2.  1.  0.  0.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]]
minz  -1
optimal trans  [[ 5.  5.  6.  6.  5.  4.  3.  2.  2.  1.  0.  0.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]]
minz  -1
/media/ula/DATADRIVE1/fos_gfp_tmaze/ctx_landmark/despeckle/m1r2_landmark_spots.tif True
optimal trans  [[7. 7. 7. 6. 6. 5. 5. 4. 4. 4. 3. 4. 4. 3. 3.]
 [4. 4. 4. 3. 3. 3. 2. 2. 2. 1. 1. 1. 1. 1. 1.]]
minz  1
optimal trans  [[7. 7. 7. 6. 6. 5. 5. 4. 4. 4. 3. 4. 4. 3. 3.]
 [4. 4. 4. 3. 3. 3. 2. 2. 2. 1. 1. 

/media/ula/DATADRIVE1/fos_gfp_tmaze/ctx_landmark/despeckle/m17r3_landmark_spots.tif True
optimal trans  [[ 8.  7.  7.  6.  6.  5.  5.  5.  4.  4.  4.  4.  4.  4.  4.  4.]
 [-4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -5. -5. -5. -5.]]
minz  0
optimal trans  [[ 8.  7.  7.  6.  6.  5.  5.  5.  4.  4.  4.  4.  4.  4.  4.  4.]
 [-4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -5. -5. -5. -5.]]
minz  0
/media/ula/DATADRIVE1/fos_gfp_tmaze/ctx_landmark/despeckle/m17r3_ctx1_spots.tif True
optimal trans  [[3. 3. 3. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1.]
 [3. 3. 3. 3. 3. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3.]]
minz  0
optimal trans  [[3. 3. 3. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1.]
 [3. 3. 3. 3. 3. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3.]]
minz  0
/media/ula/DATADRIVE1/fos_gfp_tmaze/ctx_landmark/despeckle/m17r4_landmark_spots.tif False
/media/ula/DATADRIVE1/fos_gfp_tmaze/ctx_landmark/despeckle/m17r4_ctx1_spots.tif False
/media/ula/DATADRIVE1/fos_gfp_tmaze/ctx_landmark/despeckle/m18r1_landmark_spots.ti